This notebook serves as an example of how to use the OrderParameterEnsemble class to calculate nematic order parameters, translational order parameters and structure factors. The sample simulation consists is of a coarse grain simulation of water and monoglyceride molecules (more details can be found in "files_orderparams/README.md"). 

The first step is to import the ClusterEnsemble and OrderParameterEnsemble classes as well as the numpy module to format some of the inputs.

In [ ]:
import numpy as np
from clustercode.ClusterEnsemble import ClusterEnsemble
from clustercode.OrderParameterEnsemble import OrderParameterEnsemble

The next step is to load the relevant files:
    - run.tpr: contains the simulation run information
    - init.gro: is the starting simulation configuration
    - traj.xtc: is the simulation trajectory
These can then be used to initiliase an instance of the OrderParameterEnsemble class. In the case below we specify that we only want to select the monoglyceride ('MGE') molecules for analysis.

In [ ]:
tpr  = "files_orderparams/run.tpr"
init_config = "files_orderparams/init.gro"
traj = "files_orderparams/traj.xtc"

OrderParamEns = OrderParameterEnsemble(tpr, traj, ['MGE'])

First we calculate the nematic order parameters of the initial configuration. The nematic order parameter gives information about how things are alligned in the system and is a measure of directional order, it ranges from -0.5 (antinematic) to 0 (isotropic) to 1 (nematic). The supplied trajectory file (files_orderparams/traj.xtc) has all the atoms in the box (gromacs flag -pbc atom), the molecules need to be made whole to do the nematic order parameter analysis thus we supply the parameter pbc_style='mol' to the nematic_op_analysis() method, which uses the gromacs command gmx trjconv to convert the file into the desired format (alternatively this could be done externally and a trajectory file with the gromacs flag -pbc mol could be supplied).

We see how the nematic order parameter changes from the first configuration (time: 0 ns) to the final configuration (time: 500 ns). This is expected as at time 0 ns the system is ordered randomly, whereas by at time 500 ns it has formed a lamellar phase. This can be seen visually using VMD, in files_orderparams folder run the command: vmd init.gro traj.xtc -e vmdlog. Note that the 'style' parameter denotes the whether a molecule or atom name has been specified as the selection.

In [ ]:
OrderParamEns.nematic_op_analysis(times=[0, 20000], pbc_style='mol', style='molecule')
early_system_director_list = OrderParamEns.system_director_list

In [ ]:
OrderParamEns.nematic_op_analysis(times=[480000, 500000], pbc_style='mol', style='molecule')
late_system_director_list = OrderParamEns.system_director_list

The nematic_op_analysis() method assigns a number of attributes:

In [ ]:
print("List of nematic order parameters: {}".format(OrderParamEns.nematic_op_list))
print("List of system directors: {}".format(OrderParamEns.system_director_list))
print("The mean nematic order parameter {:.3f} with standard deviation {:.3f}".format(OrderParamEns.mean_nematic_op, 
                                                                                      OrderParamEns.stdev_nematic_op))
print("The mean system director: {}".format(OrderParamEns.mean_system_director))

The nematic order parameter calculations depend on a principal molecular axis that is chosen for each molecule (or group of atoms). The two options available in the code are:
    - principal_axis='intertial': the principal inertial axis (default, used in the above calculations)
    - principal_axis='end-to-end': the end-to-end vector between the first and last atom in each AtomGroup
As can be seen below for the cases of the monoglyceride (MGE) molecules using the end-to-end vector are small compared to the principal inertial axis.

In [ ]:
OrderParamEns.nematic_op_analysis(times=[0, 20000], pbc_style='mol', style='molecule', principal_axis='end-to-end')

In [ ]:
OrderParamEns.nematic_op_analysis(times=[480000, 500000], pbc_style='mol', style='molecule', principal_axis='end-to-end')

Let's now continue to the calculation of the translational order parameter, which varies from 0 (no order) to 1 (ordered). The translational order parameter is calculated along a particular direction, often the system director obtained during the nematic order parameter calculation. The translational_op_analysis method has one mandatory parameter: the director, which can either be a list of directors one for each time or a constant value for all times. Here we see how the translational order parameter differs at the beginning and at the end of the simulation. Note that if plot=True the translational order parameter is plot versus the translational spacing for the first time that is analysed, this can be used to determine if the default search_param settings, which is a list of [min distance / Angstrom, max distance / Angstrom, # points considered] used to set the range and number of points considered when finding the optimal translation spacing, are sufficient or whether they need to be changed.

In [ ]:
OrderParamEns.translational_op_analysis(early_system_director_list, times=[0, 20000], plot=True)

In [ ]:
OrderParamEns.translational_op_analysis(late_system_director_list, times=[480000, 500000], plot=True)

The translation_op_analysis() method assigns a number of attributes:

In [ ]:
print("List of translational order parameters: {}".format(OrderParamEns.trans_op_list))
print("List of translational spacings in Angstrom: {}".format(OrderParamEns.trans_spacing_list))
print("The mean translational order parameter {:.3f} with standard deviation {:.3f}".format(OrderParamEns.mean_trans_op, 
                                                                                            OrderParamEns.stdev_trans_op))
print("The mean translational spacing {:.1f} Angstrom with standard deviation {:.1f} Angstrom".format(OrderParamEns.mean_trans_spacing, 
                                                                                                      OrderParamEns.stdev_trans_spacing))

We can change the search parameters (search_param) to check whether we considered a large enough range. We can see we double the maximum translational spacing considered the peak does not change.

In [ ]:
OrderParamEns.translational_op_analysis(late_system_director_list, times=[480000, 500000], plot=True, search_param=[0.1, 100, 1000])

As can be seen besides the translation order parameter the translational spacing (at which intervals order occurs) is also given. The translational spacing found in this case actually corresponds to the distance between the centers of mass of the MGE molecules on each side of a lamella.

Instead of specifying a system director for each frame a single one for all frames can be used and this could be set to an arbitrary value if desired. By default the translational order between the center of mass (pos_style='com') of species is determined. The translational order between atoms can also be determined (pos_style='atom'), although for our particular example this is less meaningful. Here for example we check the translational order for atoms (coarse-grain beads in the MGE molecules) along the y-axis.

In [ ]:
OrderParamEns.translational_op_analysis(np.asarray([0.0, 1.0, 0.0]), times=[480000, 500000], pos_style='atom', plot=True)

The structure factor allows one to indentify structuring within the simulation. It is best to use the plot_style='smooth' for single frames or the plot_style='scatter' for multiple to see the trends. By default the strict definition of the structure factor is used (q_style='strict'), which takes into account the wave vectors that are possible within the simulation cell considering that the boundaries are periodic. The structure factor and wave vector information can be accessed via a number of the OrderParameterEnsemble class attributes:
    - q_norm_array: numpy array of the moduli of the generated wave vectors (q)
    - Sq_array: numpy array of structure factors corresponding to each wave vector in q_norm_array
    - smooth_q_norm: numpy array of the array of the modulus of the binned wave vectors
    - smooth_Sq: numpy array of the averaged and binned structure factors for each value of q in smooth_q_norm

In [ ]:
OrderParamEns.structure_factor_analysis(times=[500000, 500000], plot_style='smooth')

In [ ]:
OrderParamEns.structure_factor_analysis(times=[450000, 500000], plot_style='scatter')

The evenly spaced peaks are indicative of lamellar behaviour. We can also try a grid search of wave vectors (the range of wave vectors considered is controlled by the variable q_min, q_max and q_step the units of which are Angtrom^(-1)). In this case the peaks become much broader as the wave vector is no longer limited by the box dimensions. It can also be useful showing it as log plot (using yscale='log') to more clearly distinguish the peaks.

In [ ]:
OrderParamEns.structure_factor_analysis(times=[500000, 500000], plot_style='scatter', q_style='grid', q_min=0.05, q_max=0.5, q_step=0.01)

In some cases it may be desired to only consider the structure factor in certain directions. These can be controlled by the active_dim parameter. By default this is equal to [1, 1, 1], meaning all dimensions are considered. Note that here the dimensions are defined as the perpendicular vectors to the simulation cell faces (the dimensions in which there is periodicity), for an orthorhombic simulation cell this would be the x, y and z directions, whereas for a triclinic box these are automatically calculated. To turn off a dimension set the respective value to in the active_dim list to 0.

In [ ]:
OrderParamEns.structure_factor_analysis(times=[500000, 500000], plot_style='scatter', q_style='grid', q_min=0.05, q_max=0.5, q_step=0.01, active_dim=[1, 1, 0])

Another setting that can be changed is the directors along which to calculate the order. By default these are taking as the ones normal to the box faces in which the periodic images occur, but they can also be set to custom variables using the directors parameter. This can be a single numpy array if the same director is applied to each frame or a list of numpy arrays one for each time frame. The numpy array must have three columns and can have upto 3 rows (one for each director).

In [ ]:
OrderParamEns.structure_factor_analysis(times=[500000, 500000], plot_style='scatter', q_style='grid', q_min=0.05, q_max=0.5, q_step=0.01, directors=OrderParamEns.mean_system_director)

The order parameter and structure factor calculations can also be performed on trajectories of clusters, a "custom trajectory". Lets take a few frames from the end of the simulaton, in which the system has formed a lamellar phase, and calculate the clusters. The clusters are defined by grouping CM and CE segments and the periodic boundary conditions are turned off. We further manipulate the trajectory by removing all clusters of less than 3000 atoms. 

In [ ]:
ClusEnsemble = ClusterEnsemble(tpr, traj, ['CM','CE'])
ClusEnsemble.cluster_analysis(cut_off=7.5, times=[470000, 490000], work_in="Atom", style="atom", pbc=False)

raw_cluster_list = ClusEnsemble.cluster_list
filtered_cluster_list = []
for frame in raw_cluster_list:
    filtered_cluster_list.append([])
    for cluster in frame:
        if len(cluster) > 3000:
            filtered_cluster_list[-1].append(cluster)

Now lets calculate the nematic order parameter for the filtered cluster trajectory (each cluster corresponds to one lamella layer). Note it is important to initialise the OrderParameterEnsemble class with the same parameters as used in the ClusterEnsemble class and to specify exactly the same times. The filtered_cluster_list is now given in the nematic_op_analysis as a custom_traj.

In [ ]:
cluster_OrderParamEns = OrderParameterEnsemble(tpr, traj, ['CM','CE'])
cluster_OrderParamEns.nematic_op_analysis(times=[470000, 490000], custom_traj=filtered_cluster_list)
cluster_system_director_list = OrderParamEns.system_director_list

Note that similar calculations can be done for the translational order parameter and structure factor, but due to the low number of clusters (only 3) the statistics will be poor. Additionally if the nematic order parameter is not high, the translational order parameter has less meaning. However, we here calculate the translational order parameters along the mean director of the MGE molecules.

In [ ]:
cluster_OrderParamEns.translational_op_analysis(directors=OrderParamEns.mean_system_director, times=[470000, 490000], custom_traj=filtered_cluster_list, plot=True)

In [ ]:
cluster_OrderParamEns.structure_factor_analysis(times=[470000, 470000], q_style='grid', custom_traj=[filtered_cluster_list[0]], plot_style="smooth", directors=OrderParamEns.mean_system_director)